In [1]:
instance = 1

In [2]:
# DERIVE THE EOMs SYMBOLICALLY ----------------------------------------------------------------------------------------------

# import libraries
import sympy as sym
import numpy as np

sym.init_printing()
from IPython.display import display #for pretty printing

# create symbolic variables

# system parameters
g = sym.symbols('g')
mb,ml1,ml2 = sym.symbols(['m_{body}','m_{leg1}','m_{leg2}']) # mass
lb,ll1,ll2 = sym.symbols(['l_{body}','l_{leg1}','l_{leg2}']) # length
Inb,Inl1,Inl2 = sym.symbols(['I_{body}','I_{leg1}','I_{leg2}']) # moment of intertia

# generalized coordinates
x,y,thb,thl,r = sym.symbols(['x','y','\\theta_{body}','\\theta_{leg}','r']) 
dx,dy,dthb,dthl,dr = sym.symbols(['\dot{x}','\dot{y}','\dot{\\theta}_{body}','\dot{\\theta}_{leg}','\dot{r}']) 
ddx,ddy,ddthb,ddthl,ddr = sym.symbols(['\ddot{x}','\ddot{y}','\ddot{\\theta}_{body}','\ddot{\\theta}_{leg}','\ddot{r}']) 

q = sym.Matrix([[x],[y],[thb],[thl],[r]])
dq = sym.Matrix([[dx],[dy],[dthb],[dthl],[dr]])
ddq = sym.Matrix([[ddx],[ddy],[ddthb],[ddthl],[ddr]])

# forces
# total joint action = actuator + rebound, but that will be dealt with elsewhere
F,tau,GRFx,GRFy = sym.symbols(['F','\\tau','G_x','G_y']) 

# STEP 1: position vectors ri = [x,y,theta] (world frame)
rb = sym.Matrix([[x],
                [y],
                [thb]])

rl1 = sym.Matrix([[x + 0.5*ll1*sym.sin(thl)],
                [y - 0.5*ll1*sym.cos(thl)],
                [thl]])

rl2 = sym.Matrix([[x + (0.5*ll1+r)*sym.sin(thl)],
                [y - (0.5*ll1+r)*sym.cos(thl)],
                [thl]])

pfoot = sym.Matrix([[x + (ll1+r)*sym.sin(thl)],
                [y - (ll1+r)*sym.cos(thl)]])

# the Jacobians
Jb = rb.jacobian(q)
Jl1 = rl1.jacobian(q)
Jl2 = rl2.jacobian(q)

# STEP 2: generate expressions for the system space velocities from the jacobians
vb = Jb*dq
vl1 = Jl1*dq
vl2 = Jl2*dq

vfoot = pfoot.jacobian(q)*dq

# STEP 3: generate expressions for the kinetic and potential energy
# mass vectors
Mb = sym.Matrix([[mb,mb,Inb]])
Ml1 = sym.Matrix([[ml1,ml1,Inl1]])
Ml2 = sym.Matrix([[ml2,ml2,Inl2]])

T = 0.5*Mb*sym.matrix_multiply_elementwise(vb,vb) + 0.5*Ml1*sym.matrix_multiply_elementwise(vl1,vl1) + 0.5*Ml2*sym.matrix_multiply_elementwise(vl2,vl2)
T = T[0]
V = mb*g*rb[1] + ml1*g*rl1[1] + ml2*g*rl2[1]


# STEP 4: calculate each term of the Lagrange equation
# term 1
Lg1 = sym.zeros(len(q),1)
for i in range(len(q)):
    dT_ddq = sym.Matrix([sym.diff(T,dq[i])]) # get partial of T in dq_i
    Lg1[i] = dT_ddq.jacobian(q)*dq + dT_ddq.jacobian(dq)*ddq #...then get time derivative of that partial

# term 3
Lg3 = sym.Matrix([T]).jacobian(q).transpose() # partial of T in q

# term 4
Lg4 = sym.Matrix([V]).jacobian(q).transpose() # partial of U in q

# STEP 5: generalized forces

# force vectors for each link
tau_b = sym.Matrix([[0],[0],[-tau]])
tau_l1 = sym.Matrix([[0],[0],[tau]])

F_l1 = sym.Matrix([[-F*sym.sin(thl)],[F*sym.cos(thl)],[0]])
F_l2 = sym.Matrix([[F*sym.sin(thl)],[-F*sym.cos(thl)],[0]])

GRF_l2 = sym.Matrix([[GRFx],[GRFy],[0.5*ll2*GRFx*sym.cos(thl)+0.5*ll2*GRFy*sym.sin(thl)]])

Q = sym.zeros(len(q),1)
for j in range(len(q)):
    Q[j] = tau_b.transpose()*Jb[:,j]+(tau_l1+F_l1).transpose()*Jl1[:,j]+(F_l2+GRF_l2).transpose()*Jl2[:,j]

# AND combine!
EOM = Lg1 - Lg3 + Lg4 - Q


In [3]:
# Lambdify
from pyomo.environ import*
from pyomo.opt import SolverFactory
from pyomo.opt import SolverStatus, TerminationCondition

func_map = {'sin':sin, 'cos':cos} 

sym_list = [g,mb,ml1,ml2,lb,ll1,ll2,Inb,Inl1,Inl2,
            x,y,thb,thl,r,
            dx,dy,dthb,dthl,dr,
            ddx,ddy,ddthb,ddthl,ddr,
            F,tau,GRFx,GRFy]

lambEOM = {}

DOFs = ['x','y','thb','thl','r']

for dof_i, dof in enumerate(DOFs):
    lambEOM.update({dof: sym.lambdify(sym_list,EOM[dof_i],modules = [func_map])})
    
lamb_footy = sym.lambdify(sym_list,pfoot[1],modules = [func_map])
lamb_dxfoot = sym.lambdify(sym_list,vfoot[0],modules = [func_map])

In [4]:
# rerun from here if you don't want to calculate the EOM's again
if 'm' in globals():
    del m # deletes the model
    
m = ConcreteModel()

# SETS-----------------------------------------------------------------------------------------------------------------------

N = 150
m.N = RangeSet(N) 

P = 3
m.P = RangeSet(P)

links = [('body',1),('leg',1),('leg',2)]
m.L = Set(dimen=2, initialize = links)

joints = ['hip','knee']
m.J = Set(initialize = joints)

DOFs = ['x','y','thb','thl','r'] # generalized coordinates
m.DOF = Set(initialize = DOFs) 

WDOFs = ['X','Y',"THETA"] # absolute coordinates 
m.WDOF = Set(initialize = WDOFs) 

signs = ['ps','ng'] # sign set for positive and negative components
m.sgn = Set(initialize = signs)

# PARAMETERS-----------------------------------------------------------------------------------------------------------------

m.g = Param(initialize = 9.81)

def get_m(n, lb, ln):
    if lb == 'body':
        return 0.9
    else: return 0.05
m.m = Param(m.L, initialize = get_m) # mass of links

def get_len(n, lb, ln):
    if lb == 'body':
        return 0.6
    else: return 0.3
m.len = Param(m.L, initialize = get_len) # length of links

def calculate_In(m, lb, ln): 
    l = (lb,ln)
    # yes, that does mean you have to rebuild the tuple inside the function. Yes, it is dumb.
    return m.m[l]*m.len[l]**2/12 
m.In = Param(m.L, initialize = calculate_In) # moment of inertia

mbody = sum(m.m[l] for l in links)
BW = mbody*m.g.value

# VARIABLES -----------------------------------------------------------------------------------------------------------------

# system coordinates
m.q = Var(m.N, m.P, m.DOF) # position
m.dq = Var(m.N, m.P, m.DOF) # velocity
m.ddq = Var(m.N, m.P, m.DOF) # acceleration

m.GRF = Var(m.N, m.P, m.WDOF, m.sgn, bounds = (0.0,None)) # ground reaction forces

m.force_a = Var(m.N, m.J) # actuator 
m.force_r = Var(m.N, m.P, m.J, m.sgn, bounds = (0,1)) # rebound 

# bound variables
for n in range(1,N+1):
    for p in range(1,P+1):
        m.q[n,p,'y'].setlb(0.0)
        m.q[n,p,'thb'].setlb(-np.pi/2)
        m.q[n,p,'thb'].setub(np.pi/2)
        m.GRF[n,p,'Y','ps'].setub(3.0)
            
# TIME AND INTEGRATION ---------------------------------------------------------------------------------------------------

# variable timestep
m.hm = Param(initialize = 0.02, mutable = True) # master timestep
m.h = Var(m.N, bounds = (0.8,1.2))

# Integration constraints 
radau_mat = [[0.19681547722366, -0.06553542585020, 0.02377097434822],
 [0.39442431473909, 0.29207341166523, -0.04154875212600],
 [0.37640306270047, 0.51248582618842, 0.11111111111111]]

# aux variable to assist with scaling
m.acc = Var(m.N, m.P, m.DOF)
m.vel = Var(m.N, m.P, m.DOF)

# RADAU
def get_acc(m,n,p,dof):
    if n == 1:
        return Constraint.Skip
    if n > 1:
        return m.acc[n,p,dof] == m.hm*m.h[n]*sum([radau_mat[p-1][pp-1]*m.ddq[n,pp,dof] for pp in range(1,P+1)])
m.get_acc = Constraint(m.N, m.P, m.DOF, rule = get_acc)

def get_vel(m,n,p,dof):
    if n == 1:
        return Constraint.Skip
    if n > 1:
        return m.vel[n,p,dof] == m.hm*m.h[n]*sum([radau_mat[p-1][pp-1]*m.dq[n,pp,dof] for pp in range(1,P+1)])
m.get_vel = Constraint(m.N, m.P, m.DOF, rule = get_vel)

def Radau_p(m,n,p,dof):
    if n == 1:
        return Constraint.Skip
    if n > 1:
        return m.q[n,p,dof] == m.q[n-1,P,dof] + m.vel[n,p,dof]
m.integrate_p3 = Constraint(m.N, m.P, m.DOF, rule = Radau_p)

def Radau_v(m,n,p,dof):
    if n == 1:
        return Constraint.Skip
    if n > 1:
        return m.dq[n,p,dof] == m.dq[n-1,P,dof] + m.acc[n,p,dof]
m.integrate_v3 = Constraint(m.N, m.P, m.DOF, rule = Radau_v)

# BACKWARD EULER
# Integration constraints 
def BwEuler_p(m,n,dof): # for positions
    if n > 1:
        return m.q[n,P,dof] == m.q[n-1,P,dof] + m.hm*m.h[n]*m.dq[n,P,dof]
    else:
        return Constraint.Skip #use this to leave out members of a set that the constraint doesn't apply to
m.integrate_p1 = Constraint(m.N, m.DOF, rule = BwEuler_p)

def BwEuler_v(m,n,dof): # for velocities
    if n > 1:
        return m.dq[n,P,dof] == m.dq[n-1,P,dof] + m.hm*m.h[n]*m.ddq[n,P,dof]
    else:
        return Constraint.Skip 
m.integrate_v1 = Constraint(m.N, m.DOF, rule = BwEuler_v)

# GROUND INTERACTIONS -------------------------------------------------------------------------------------------------------

ground_constraints = ['contact','friction','slip_ps','slip_ng'] 
m.ground_constraints = Set(initialize = ground_constraints) # set for indexing ground-related penalties

# variables
m.footy = Var(m.N, m.P, bounds = (0.0,None)) # foot position
m.footdx = Var(m.N, m.P, m.sgn, bounds = (0.0,None)) # foot velocity
m.friction_cone = Var(m.N, m.P, m.sgn, bounds = (0.0,None))
m.ground_penalty = Var(m.N, m.ground_constraints, bounds = (0.0,None))

m.mu_k = Param(initialize = 1.2, mutable = True)
m.mu_s = Param(initialize = 2.4, mutable = True)
m.a_k = Var(m.N, m.P, bounds = (0.0, m.mu_s.value))

# constraints: aux variables
def def_footy(m,n,p):
    if n == 1 and p < P:
        return Constraint.Skip
    else:
        F_in = 0
        tau_in = 0
        Gx_in = 0
        Gy_in = 0
        var_list = [m.g,m.m[('body',1)],m.m[('leg',1)],m.m[('leg',2)],
        m.len[('body',1)],m.len[('leg',1)],m.len[('leg',2)],
        m.In[('body',1)],m.In[('leg',1)],m.In[('leg',2)],
        m.q[n,p,'x'],m.q[n,p,'y'],m.q[n,p,'thb'],m.q[n,p,'thl'],m.q[n,p,'r'],
        m.dq[n,p,'x'],m.dq[n,p,'y'],m.dq[n,p,'thb'],m.dq[n,p,'thl'],m.dq[n,p,'r'],
        m.ddq[n,p,'x'],m.ddq[n,p,'y'],m.ddq[n,p,'thb'],m.ddq[n,p,'thl'],m.ddq[n,p,'r'],
        F_in,tau_in,Gx_in,Gy_in]
        
        return m.footy[n,p] == lamb_footy(*var_list)
m.def_footy = Constraint(m.N, m.P, rule = def_footy)

def def_friction_cone(m,n,p):
    if n == 1:
        return Constraint.Skip
    return m.friction_cone[n,p,'ps'] - m.friction_cone[n,p,'ng'] == m.a_k[n,p] - m.mu_k
m.def_friction_cone = Constraint(m.N, m.P, rule = def_friction_cone)

def def_footdx(m,n,p):
    if n == 1 and p < P:
        return Constraint.Skip
    else:
        F_in = 0
        tau_in = 0
        Gx_in = 0
        Gy_in = 0
        var_list = [m.g,m.m[('body',1)],m.m[('leg',1)],m.m[('leg',2)],
        m.len[('body',1)],m.len[('leg',1)],m.len[('leg',2)],
        m.In[('body',1)],m.In[('leg',1)],m.In[('leg',2)],
        m.q[n,p,'x'],m.q[n,p,'y'],m.q[n,p,'thb'],m.q[n,p,'thl'],m.q[n,p,'r'],
        m.dq[n,p,'x'],m.dq[n,p,'y'],m.dq[n,p,'thb'],m.dq[n,p,'thl'],m.dq[n,p,'r'],
        m.ddq[n,p,'x'],m.ddq[n,p,'y'],m.ddq[n,p,'thb'],m.ddq[n,p,'thl'],m.ddq[n,p,'r'],
        F_in,tau_in,Gx_in,Gy_in]
        
        return m.footdx[n,p,'ps']-m.footdx[n,p,'ng'] == lamb_dxfoot(*var_list)
m.def_footdx = Constraint(m.N, m.P, rule = def_footdx)

def def_Gx(m,n,p):
    if n == 1:
        return Constraint.Skip
    return m.GRF[n,p,'X','ps'] + m.GRF[n,p,'X','ng'] == m.a_k[n,p]*m.GRF[n,p,'Y','ps']
m.def_Gx = Constraint(m.N, m.P, rule = def_Gx)

# constraints: complementarity
m.ground_varA = Var(m.N, m.ground_constraints)
m.ground_varB = Var(m.N, m.ground_constraints)

def def_ground_varA(m,n,gc):
    if n == 1:
        return Constraint.Skip
    if gc == 'contact':
        if n < N:
            return m.ground_varA[n,gc] == sum(m.footy[n+1,p] for p in range(1,P+1))
        else: 
            return m.ground_varA[n,gc] == sum(m.footy[n,p] for p in range(1,P+1))
    if gc == 'friction':
        return m.ground_varA[n,gc] == sum(m.footdx[n,p,'ps']+m.footdx[n,p,'ng'] for p in range(1,P+1))
    if gc == 'slip_ps':
        return m.ground_varA[n,gc] == sum(m.footdx[n,p,'ps'] for p in range(1,P+1))
    if gc == 'slip_ng':
        return m.ground_varA[n,gc] == sum(m.footdx[n,p,'ng'] for p in range(1,P+1))
m.def_ground_varA = Constraint(m.N, m.ground_constraints, rule = def_ground_varA)

def def_ground_varB(m,n,gc):
    if n == 1:
        return Constraint.Skip
    if gc == 'contact':
        return m.ground_varB[n,gc] == sum(BW*m.GRF[n,p,'Y','ps'] for p in range(1,P+1))
    if gc == 'friction':
        return m.ground_varB[n,gc] == sum(BW*m.friction_cone[n,p,'ps']+BW*m.friction_cone[n,p,'ng'] for p in range(1,P+1))
    if gc == 'slip_ps':
        return m.ground_varB[n,gc] == sum(BW*m.GRF[n,p,'X','ps'] for p in range(1,P+1))
    if gc == 'slip_ng':
        return m.ground_varB[n,gc] == sum(BW*m.GRF[n,p,'X','ng'] for p in range(1,P+1))
m.def_ground_varB = Constraint(m.N, m.ground_constraints, rule = def_ground_varB)

# complementarity
def ground_comp(m,n,gc):
    if n == 1:
        return Constraint.Skip
    return m.ground_penalty[n,gc] == m.ground_varA[n,gc]*m.ground_varB[n,gc]
m.ground_comp = Constraint(m.N, m.ground_constraints, rule = ground_comp)

# HARD JOINT STOPS ----------------------------------------------------------------------------------------------------------

joint_constraints = ['up','lo'] # set of joint penalties
m.joint_constraints = Set(initialize = joint_constraints)

# variables
m.joint_penalty = Var(m.N, m.J, m.joint_constraints, bounds = (0.0,None))
m.joint_varA = Var(m.N, m.J, m.joint_constraints)
m.joint_varB = Var(m.N, m.J, m.joint_constraints)

m.jointp = Var(m.N, m.P, m.J)
m.jointv = Var(m.N, m.P, m.J)
m.power = Var(m.N, m.P, m.J)

for n in range(1,N+1):
    for p in range(1,P+1):
        m.jointp[n,p,'hip'].setlb(-np.pi/2)
        m.jointp[n,p,'hip'].setub(np.pi/2)
        m.jointp[n,p,'knee'].setlb(0)
        m.jointp[n,p,'knee'].setub(0.3)
        
def def_jointp(m,n,p,j):
    if n == 1 and p < P:
        return Constraint.Skip
    if j == 'hip':
        return m.jointp[n,p,j] == m.q[n,p,'thl'] - m.q[n,p,'thb']
    if j == 'knee':
        return m.jointp[n,p,j] == m.q[n,p,'r']
m.def_jointp = Constraint(m.N, m.P, m.J, rule = def_jointp)

def def_jointv(m,n,p,j):
    if n == 1 and p < P:
        return Constraint.Skip
    if j == 'hip':
        return m.jointv[n,p,j] == m.dq[n,p,'thl'] - m.dq[n,p,'thb']
    if j == 'knee':
        return m.jointv[n,p,j] == m.dq[n,p,'r']
m.def_jointv = Constraint(m.N, m.P, m.J, rule = def_jointv)

def def_power(m,n,p,j):
    if n == 1:
        return Constraint.Skip
    else:
        return m.power[n,p,j] == 9.81*m.force_a[n,j]*m.jointv[n,p,j]
m.def_power = Constraint(m.N, m.P, m.J, rule = def_power)

def def_joint_varA(m,n,j,jc):
    if n == 1:
        return Constraint.Skip
    if jc == 'up':
        if n < N:
            return m.joint_varA[n,j,jc] == sum(m.jointp[3,P,j].ub - m.jointp[n+1,p,j] for p in range(1,P+1))
        else:
            return m.joint_varA[n,j,jc] == sum(m.jointp[3,P,j].ub - m.jointp[n,p,j] for p in range(1,P+1))
    if jc == 'lo':
        if n < N:
            return m.joint_varA[n,j,jc] == sum(m.jointp[n+1,p,j] - m.jointp[3,P,j].lb for p in range(1,P+1))
        else:
            return m.joint_varA[n,j,jc] == sum(m.jointp[n,p,j] - m.jointp[3,P,j].lb for p in range(1,P+1))
m.def_joint_varA = Constraint(m.N, m.J, m.joint_constraints, rule = def_joint_varA)

# define aux variables
def def_joint_varB(m,n,j,jc):
    if n == 1:
        return Constraint.Skip
    if jc == 'up':
        return m.joint_varB[n,j,jc] == sum(BW*m.force_r[n,p,j,'ng'] for p in range(1,P+1))
    if jc == 'lo':
        return m.joint_varB[n,j,jc] == sum(BW*m.force_r[n,p,j,'ps'] for p in range(1,P+1))
m.def_joint_varB = Constraint(m.N, m.J, m.joint_constraints, rule = def_joint_varB)

# complementarity
def joint_comp(m,n,j,jc):
    if n == 1:
        return Constraint.Skip
    return m.joint_penalty[n,j,jc] == m.joint_varA[n,j,jc]*m.joint_varB[n,j,jc]
m.joint_comp = Constraint(m.N, m.J, m.joint_constraints, rule = joint_comp)

# stop ends of links from going through the floor
m.jointy = Var(m.N, m.P, m.J, bounds = (0.0,None))

# DYNAMICS ----------------------------------------------------------------------------------------------------------------
S = BW

def dynamics(m,n,p,dof):
    if n == 1:
        return Constraint.Skip
    else:
        F_in = m.force_a[n,'knee'] + m.force_r[n,p,'knee','ps'] - m.force_r[n,p,'knee','ng']
        tau_in = m.force_a[n,'hip'] + m.force_r[n,p,'hip','ps'] - m.force_r[n,p,'hip','ng']
        Gx_in = m.GRF[n,p,'X','ps'] - m.GRF[n,p,'X','ng']
        Gy_in = m.GRF[n,p,'Y','ps']
        var_list = [m.g,m.m[('body',1)],m.m[('leg',1)],m.m[('leg',2)],
        m.len[('body',1)],m.len[('leg',1)],m.len[('leg',2)],
        m.In[('body',1)],m.In[('leg',1)],m.In[('leg',2)],
        m.q[n,p,'x'],m.q[n,p,'y'],m.q[n,p,'thb'],m.q[n,p,'thl'],m.q[n,p,'r'],
        m.dq[n,p,'x'],m.dq[n,p,'y'],m.dq[n,p,'thb'],m.dq[n,p,'thl'],m.dq[n,p,'r'],
        m.ddq[n,p,'x'],m.ddq[n,p,'y'],m.ddq[n,p,'thb'],m.ddq[n,p,'thl'],m.ddq[n,p,'r'],
        S*F_in,S*tau_in,S*Gx_in,S*Gy_in]
    
    return lambEOM[dof](*var_list) == 0
m.dynamics = Constraint(m.N, m.P, m.DOF, rule = dynamics)

# OBJECTIVE --------------------------------------------------------------------------------------------------------------
def MinPenalty(m):
    penalty_sum = 0
    for n in range(1,N+1):
        for gc in ground_constraints:
            penalty_sum += m.ground_penalty[n,gc]
        for j in joints:
            for jc in joint_constraints:
                penalty_sum += m.joint_penalty[n,j,jc]
    return penalty_sum
m.MinPenalty = Objective(rule = MinPenalty)

In [5]:
def init_opt(): # initializes solver
    opt = SolverFactory('ipopt')
    #opt.options["linear_solver"] = 'ma97'
    opt.options["print_level"] = 5 # prints a log with each iteration (you want to this - it's the only way to see progress.)
    opt.options["max_iter"] = 30000 # maximum number of iterations
    opt.options["max_cpu_time"] = 3600 # maximum cpu time in seconds
    opt.options["Tol"] = 1e-6 # the tolerance for feasibility. Considers constraints satisfied when they're within this margin.
    
    opt.options["OF_acceptable_obj_change_tol"] = 1e-4
    opt.options["OF_ma86_scaling"] = 'none'
    
    return opt

In [6]:
def euler_mode(m):
    m.integrate_p1.activate()
    m.integrate_v1.activate()
    
    # deactivate constraints
    m.get_acc.deactivate()
    m.get_vel.deactivate()
    m.integrate_p3.deactivate()
    m.integrate_v3.deactivate()

    for n in range(2,N+1):
        for p in range(1,P):
            m.def_footy[n,p].deactivate()
            m.def_footdx[n,p].deactivate()
            m.def_Gx[n,p].deactivate()
            m.def_friction_cone[n,p].deactivate()
            for j in joints:
                m.def_jointp[n,p,j].deactivate()
                m.def_jointv[n,p,j].deactivate()
                m.def_power[n,p,j].deactivate()
            for dof in DOFs:
                m.dynamics[n,p,dof].deactivate()

#     fix variables
    for n in range(2,N+1):
        for p in range(1,P):
            for dof in DOFs:
                m.q[n,p,dof].fix(0.0)
                m.dq[n,p,dof].fix(0.0)
                m.ddq[n,p,dof].fix(0.0)
            
            m.a_k[n,p].fix(0.0)
            for dof in WDOFs:
                for sgn in signs:
                    m.GRF[n,p,dof,sgn].fix(0.0)
                    m.footy[n,p].fix(0.0)
                    m.footdx[n,p,sgn].fix(0.0)
                    m.friction_cone[n,p,sgn].fix(0.0)
            for j in joints:
                for sgn in signs:
                    m.force_r[n,p,j,sgn].fix(0.0)
                    m.jointp[n,p,j].fix(0.0)
                    m.jointv[n,p,j].fix(0.0)
                    m.power[n,p,j].fix(0.0)                                  
    return m

def radau_mode(m):
    m.integrate_p1.deactivate()
    m.integrate_v1.deactivate()
    
    # deactivate constraints
    m.get_acc.activate()
    m.get_vel.activate()
    m.integrate_p3.activate()
    m.integrate_v3.activate()

    m.def_footy.activate()
    m.def_footdx.activate()
    m.def_Gx.activate()
    m.def_friction_cone.activate()
    m.def_jointp.activate()
    m.def_jointv.activate()
    m.def_power.activate()
    m.dynamics.activate()

#   unfix variables
    for n in range(2,N+1):
        for p in range(1,P):
            for dof in DOFs:
                m.q[n,p,dof].unfix()
                m.dq[n,p,dof].unfix()
                m.ddq[n,p,dof].unfix()
            
            m.a_k[n,p].unfix()
            for dof in WDOFs:
                for sgn in signs:
                    m.GRF[n,p,dof,sgn].unfix()
                    m.footy[n,p].unfix()
                    m.footdx[n,p,sgn].unfix()
                    m.friction_cone[n,p,sgn].unfix()
            for j in joints:
                for sgn in signs:
                    m.force_r[n,p,j,sgn].unfix()
                    m.jointp[n,p,j].unfix()
                    m.jointv[n,p,j].unfix()
                    m.power[n,p,j].unfix()                                 
    return m

def init_midpoints(m):
    for n in range(2,N+1):
        for p in range(1,P):
            for dof in DOFs:
                m.q[n,p,dof].value = m.q[n,P,dof].value
                m.dq[n,p,dof].value = m.dq[n,P,dof].value
                m.ddq[n,p,dof].value = m.ddq[n,P,dof].value
            
            m.a_k[n,p].value = m.a_k[n,P].value
            for dof in WDOFs:
                for sgn in signs:
                    m.GRF[n,p,dof,sgn].value = m.GRF[n,P,dof,sgn].value
                    m.footy[n,p].value = m.footy[n,P].value
                    m.footdx[n,p,sgn].value = m.footdx[n,P,sgn].value
                    m.friction_cone[n,p,sgn].value = m.friction_cone[n,P,sgn].value
            for j in joints:
                for sgn in signs:
                    m.force_r[n,p,j,sgn].value = m.force_r[n,P,j,sgn].value
                    m.jointp[n,p,j].value = m.jointp[n,P,j].value
                    m.jointv[n,p,j].value = m.jointv[n,P,j].value
                    m.power[n,p,j].value = m.power[n,P,j].value                                
    return m

In [7]:
# 3 second sprint from rest
T = 3.0
m.hm = T/N

# initial condition
for dof in DOFs:
    m.dq[1,P,dof].fix(0)
    
m.q[1,P,'x'].fix(0)
m.q[1,P,'thb'].fix(0)
m.q[1,P,'thl'].fix(0)
m.q[1,P,'r'].fix(0.3)

# final condition
m.q[N,P,'x'].setlb(10)

# max height (so the overpowered model can't just leap the whole way :P)
for n in range(1,N+1):
    for p in range(1,P+1):
        m.q[n,p,'y'].setub(1)

In [8]:
# STAGE 1 euler solve
m = euler_mode(m)
opt = init_opt()
results = opt.solve(m, tee = True)

Ipopt 3.12.13: print_level=5
max_iter=30000
max_cpu_time=3600
Tol=1e-06
option_file_name=/tmp/tmptylaen3m_ipopt.opt

Using option file "/tmp/tmptylaen3m_ipopt.opt".


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:    27713
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:     8642

Total number of variables............................:     9105
                     variables with only lowe

  76  2.2166615e+02 6.73e+00 8.28e+02  -1.0 1.13e+01   2.0 4.72e-04 5.68e-03h  1
  77  2.2356153e+02 6.72e+00 1.94e+03  -1.0 1.05e+01   2.4 5.49e-03 2.16e-03h  1
  78  2.2667453e+02 6.70e+00 1.72e+03  -1.0 1.19e+01   1.9 1.50e-03 3.53e-03h  1
  79  2.2697879e+02 6.69e+00 3.36e+03  -1.0 1.04e+01   2.3 5.92e-03 3.42e-04h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  80  2.3300652e+02 6.65e+00 2.76e+03  -1.0 1.55e+01   1.8 3.92e-04 6.72e-03h  1
  81  2.3468578e+02 6.64e+00 5.11e+03  -1.0 1.06e+01   2.3 1.09e-02 1.88e-03h  1
  82  2.3703995e+02 6.62e+00 4.00e+03  -1.0 1.04e+01   2.7 1.09e-03 2.64e-03h  1
  83  2.3768488e+02 6.61e+00 6.07e+03  -1.0 1.08e+01   2.2 9.85e-03 7.11e-04h  1
  84  2.4514431e+02 6.56e+00 9.86e+02  -1.0 1.04e+01   2.7 2.53e-04 8.28e-03h  1
  85  2.4703390e+02 6.54e+00 3.59e+03  -1.0 1.34e+01   2.2 1.50e-02 2.07e-03h  1
  86  2.4870231e+02 6.53e+00 4.99e+03  -1.0 1.04e+01   2.6 4.37e-03 1.83e-03h  1
  87  2.5280084e+02 6.50e+00

 169  6.2608017e+02 4.50e+00 3.60e+05  -1.0 1.37e+01   2.7 7.40e-03 2.69e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 170  6.3225846e+02 4.47e+00 3.50e+05  -1.0 1.92e+01   3.2 3.63e-05 5.50e-03h  1
 171  6.3988391e+02 4.44e+00 3.17e+05  -1.0 1.10e+01   3.6 7.04e-04 6.86e-03h  1
 172  6.5004187e+02 4.40e+00 1.97e+05  -1.0 1.09e+01   4.0 8.49e-04 9.15e-03h  1
 173  6.5348604e+02 4.39e+00 2.25e+05  -1.0 1.15e+01   3.5 9.78e-03 3.09e-03h  1
 174  6.5902783e+02 4.37e+00 5.48e+05  -1.0 1.08e+01   4.0 3.09e-02 5.00e-03h  1
 175  6.5962322e+02 4.37e+00 5.58e+05  -1.0 5.47e+01   3.5 3.83e-03 5.08e-04h  1
 176  6.6811642e+02 4.33e+00 7.73e+05  -1.0 1.08e+01   3.9 2.80e-02 7.58e-03h  1
 177  6.6874512e+02 4.33e+00 1.23e+06  -1.0 1.10e+01   4.3 1.59e-02 5.47e-04h  1
 178  6.7289310e+02 4.31e+00 1.23e+06  -1.0 1.08e+01   3.9 3.82e-03 3.57e-03h  1
 179  6.8005182e+02 4.29e+00 1.08e+06  -1.0 1.10e+01   4.3 5.89e-04 6.19e-03h  1
iter    objective    inf_pr 

 261  8.3279639e+02 3.76e+00 1.42e+06  -1.0 1.23e+01   4.9 1.04e-04 1.83e-04h  1
 262  8.3302900e+02 3.76e+00 1.42e+06  -1.0 2.39e+01   4.4 2.72e-04 1.93e-04h  1
 263  8.4082972e+02 3.74e+00 7.34e+05  -1.0 1.22e+01   4.8 2.27e-04 6.57e-03h  1
 264  8.4091043e+02 3.74e+00 4.73e+06  -1.0 1.23e+01   5.3 1.34e-02 6.82e-05h  1
 265  8.4324111e+02 3.73e+00 4.50e+06  -1.0 1.22e+01   4.8 7.32e-05 1.96e-03h  1
 266  8.4326917e+02 3.73e+00 4.77e+06  -1.0 1.23e+01   5.2 8.53e-04 2.37e-05h  1
 267  8.4561346e+02 3.72e+00 4.57e+06  -1.0 1.21e+01   4.7 1.18e-04 1.97e-03h  1
 268  8.4564136e+02 3.72e+00 4.89e+06  -1.0 1.23e+01   5.2 1.13e-03 2.36e-05h  1
 269  8.4726634e+02 3.72e+00 4.77e+06  -1.0 1.20e+01   4.7 7.64e-05 1.37e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 270  8.4730129e+02 3.72e+00 5.30e+06  -1.0 1.22e+01   5.1 2.06e-03 2.95e-05h  1
 271  8.4775019e+02 3.71e+00 5.47e+06  -1.0 1.20e+01   4.6 2.72e-03 3.74e-04h  1
 272  8.5112907e+02 3.70e+00

 366  2.1113193e+03 6.60e+00 2.32e+01  -1.0 1.03e+03    -  3.07e-02 1.09e-01f  1
 367  2.1113198e+03 6.60e+00 7.69e+03  -1.0 2.62e+00   4.8 2.18e-02 1.21e-04h  1
 368  2.1113232e+03 6.60e+00 6.44e+04  -1.0 2.63e+00   4.3 2.09e-02 3.58e-04h  1
 369  2.1112697e+03 6.58e+00 8.64e+04  -1.0 2.65e+00   3.9 1.77e-02 2.17e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 370  2.1111768e+03 6.57e+00 9.93e+04  -1.0 2.60e+00   3.4 2.70e-02 2.56e-03h  1
 371  2.1111543e+03 6.55e+00 1.04e+05  -1.0 2.89e+00   2.9 2.75e-02 1.84e-03h  1
 372  2.1113833e+03 6.54e+00 1.05e+05  -1.0 3.55e+00   2.4 1.32e-02 1.71e-03h  1
 373  2.1127028e+03 6.52e+00 1.05e+05  -1.0 5.55e+00   2.0 6.18e-03 3.68e-03h  1
 374  2.1149146e+03 6.49e+00 1.04e+05  -1.0 6.89e+00   1.5 1.26e-02 4.29e-03h  1
 375  2.1170982e+03 6.46e+00 1.03e+05  -1.0 6.43e+00   1.9 3.30e-02 4.70e-03h  1
 376  2.1258716e+03 6.37e+00 1.01e+05  -1.0 7.74e+00   1.4 2.08e-02 1.41e-02h  1
 377  2.1268295e+03 6.36e+00

 459  2.4643970e+03 1.38e-02 4.76e+01  -1.7 1.23e+00  -0.2 4.58e-01 4.19e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 460  2.4290825e+03 2.02e-02 4.48e+01  -1.7 3.39e+00  -0.7 1.66e-01 1.48e-01f  1
 461  2.3998209e+03 1.76e-02 2.81e+02  -1.7 1.22e+00  -0.3 7.36e-01 3.38e-01f  1
 462  2.3615016e+03 2.34e-02 1.86e+02  -1.7 3.83e+00  -0.8 1.43e-01 1.91e-01f  1
 463  2.3409564e+03 1.81e-02 3.65e+02  -1.7 1.26e+00  -0.3 9.57e-01 3.17e-01f  1
 464  2.2958847e+03 3.21e-02 2.12e+02  -1.7 3.98e+00  -0.8 4.82e-01 2.58e-01f  1
 465  2.2674391e+03 2.50e-02 3.58e+02  -1.7 1.40e+00  -0.4 8.48e-01 4.52e-01f  1
 466  2.2189246e+03 5.39e-02 2.64e+02  -1.7 4.69e+00  -0.9 2.41e-01 2.54e-01f  1
 467  2.1966030e+03 4.22e-02 1.66e+02  -1.7 1.66e+00  -0.4 1.94e-01 3.20e-01f  1
 468  2.1731180e+03 4.74e-02 1.47e+02  -1.7 5.50e+00  -0.9 1.13e-01 1.12e-01f  1
 469  2.1568165e+03 4.01e-02 9.89e+01  -1.7 1.80e+00  -0.5 6.01e-01 2.31e-01f  1
iter    objective    inf_pr 

 551  5.6479668e+02 4.31e-02 3.22e+05  -2.5 4.50e+01  -1.7 1.05e-02 2.62e-02f  1
 552  5.5157575e+02 2.29e-02 3.53e+05  -2.5 9.99e-01  -0.4 9.55e-01 4.85e-01f  1
 553  5.3939261e+02 2.01e-02 2.62e+05  -2.5 3.01e+00  -0.8 7.08e-01 2.79e-01f  1
 554  5.2592908e+02 2.54e-02 2.30e+05  -2.5 1.00e+01  -1.3 5.24e-02 1.22e-01f  1
 555  5.1399078e+02 2.30e-02 1.67e+05  -2.5 3.27e+00  -0.9 2.81e-01 2.73e-01f  1
 556  5.0697631e+02 2.46e-02 1.57e+05  -2.5 1.81e+01  -1.4 1.07e-02 6.16e-02f  1
 557  5.0529492e+02 1.97e-02 1.26e+05  -2.5 4.22e-01  -0.0 3.38e-01 2.00e-01f  1
 558  5.0172499e+02 1.61e-02 1.01e+05  -2.5 1.28e+00  -0.5 4.25e-01 2.02e-01f  1
 559  4.9521993e+02 1.57e-02 8.64e+04  -2.5 3.83e+00  -1.0 1.30e-01 1.44e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 560  4.8840159e+02 1.77e-02 8.19e+04  -2.5 1.24e+01  -1.5 9.57e-02 5.24e-02f  1
 561  4.8241685e+02 1.71e-02 7.14e+04  -2.5 4.39e+00  -1.0 2.94e-01 1.30e-01f  1
 562  4.7577705e+02 1.24e-02

 644  1.4562518e+02 2.19e-02 9.32e+01  -2.5 1.13e+01  -1.8 1.13e-01 4.34e-02f  1
 645  1.4202030e+02 2.08e-02 1.57e+02  -2.5 3.77e+00  -1.4 4.76e-01 3.05e-01f  1
 646  1.3965587e+02 3.05e-02 7.09e+01  -2.5 1.41e+01  -1.9 3.18e-02 6.63e-02f  1
 647  1.3770865e+02 3.24e-02 6.31e+02  -2.5 3.97e+00  -1.4 9.24e-01 1.88e-01f  1
 648  1.3653490e+02 3.21e-02 7.65e+02  -2.5 1.10e+01  -1.9 1.80e-01 4.50e-02f  1
 649  1.3146423e+02 3.80e-02 3.81e+02  -2.5 4.20e+00  -1.5 5.04e-01 5.03e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 650  1.2678524e+02 5.67e-02 2.66e+02  -2.5 1.07e+01  -2.0 1.45e-01 1.91e-01f  1
 651  1.2601742e+02 5.24e-02 5.67e+02  -2.5 4.37e+00  -1.5 3.33e-01 8.49e-02f  1
 652  1.2601795e+02 4.73e-02 7.47e+02  -2.5 5.39e-02   0.7 7.03e-01 9.62e-02h  1
 653  1.2595404e+02 2.74e-05 8.90e-01  -2.5 1.03e-01   0.2 1.00e+00 1.00e+00h  1
 654  1.2543780e+02 2.62e-04 2.46e+00  -2.5 2.28e-01  -0.3 1.00e+00 1.00e+00f  1
 655  1.2395626e+02 2.56e-03

 737  2.5398416e+01 2.50e-01 1.65e+00  -2.5 2.68e+01  -3.7 6.85e-01 4.39e-01f  1
 738  2.5368820e+01 2.50e-01 1.59e+00  -2.5 3.21e+04  -4.2 2.27e-04 4.04e-04f  1
 739  2.4705878e+01 2.71e-01 4.34e+00  -2.5 2.89e+01  -3.8 5.50e-01 2.87e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 740  2.3972046e+01 1.70e-01 7.11e-01  -2.5 1.08e+01  -3.4 8.32e-01 8.84e-01h  1
 741  2.3660300e+01 2.03e-02 4.41e-02  -2.5 3.98e+00  -2.9 1.00e+00 1.00e+00h  1
 742  2.2839260e+01 1.56e-01 3.65e-01  -2.5 1.19e+01  -3.4 1.00e+00 9.42e-01f  1
 743  2.1343443e+01 6.28e-01 3.34e+00  -2.5 3.49e+01  -3.9 9.20e-01 6.13e-01f  1
 744  2.0476919e+01 2.02e-01 3.41e+00  -2.5 1.31e+01  -3.5 6.12e-01 1.00e+00h  1
 745  1.9185901e+01 5.16e-01 1.62e+00  -2.5 3.79e+01  -3.9 5.37e-01 5.31e-01f  1
 746  1.8393144e+01 2.04e-01 1.14e+00  -2.5 1.42e+01  -3.5 1.00e+00 7.92e-01h  1
 747  1.7992499e+01 2.59e-02 9.54e-02  -2.5 5.24e+00  -3.1 1.00e+00 1.00e+00h  1
 748  1.7111616e+01 1.80e-01

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 830  4.2610022e-03 5.20e-04 8.92e-03  -5.7 4.86e-01  -5.7 1.00e+00 1.00e+00h  1
 831  4.2609932e-03 4.82e-04 9.01e-02  -5.7 8.35e-01  -6.1 3.84e-01 1.92e-01h  2
 832  4.2611987e-03 1.14e-06 1.64e-05  -5.7 9.75e-03  -3.0 1.00e+00 1.00e+00h  1
 833  5.6829201e-04 4.45e-06 2.18e+01  -7.0 2.58e-02  -3.5 1.00e+00 8.76e-01f  1
 834  1.4389290e-04 1.24e-06 2.58e+00  -7.0 9.87e-03  -3.1 1.00e+00 9.39e-01f  1
 835  1.4950837e-04 1.89e-06 2.09e+00  -7.0 1.66e-02  -3.5 1.00e+00 7.71e-01f  1
 836  1.4852354e-04 5.82e-08 3.29e-04  -7.0 1.68e-03  -4.0 1.00e+00 1.00e+00f  1
 837  1.4854353e-04 1.44e-08 6.75e-06  -7.0 2.13e-03  -4.5 1.00e+00 1.00e+00h  1
 838  1.4854133e-04 1.11e-07 4.43e-05  -7.0 6.23e-03  -5.0 1.00e+00 1.00e+00h  1
 839  1.4853594e-04 7.21e-07 3.21e-04  -7.0 1.79e-02  -5.4 1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 840  1.4852383e-04 5.34e-06

In [9]:
# STAGE 2 radau solve
m = radau_mode(m)
results = opt.solve(m, tee = True)

Ipopt 3.12.13: print_level=5
max_iter=30000
max_cpu_time=3600
Tol=1e-06
option_file_name=/tmp/tmpytj1ux90_ipopt.opt

Using option file "/tmp/tmpytj1ux90_ipopt.opt".


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:    89083
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:    23542

Total number of variables............................:    23707
                     variables with only lowe

  76  1.7044204e+03 3.87e-01 2.08e+03  -1.0 4.65e+00  -1.0 5.96e-01 2.42e-01f  1
  77  1.6243281e+03 2.70e-01 6.34e+02  -1.0 1.11e+01  -1.4 1.92e-01 4.35e-01f  1
  78  1.6034769e+03 2.65e-01 6.24e+02  -1.0 7.85e+01  -1.9 1.12e-02 4.09e-02f  1
  79  1.5745683e+03 2.37e-01 1.25e+03  -1.0 1.15e+01  -1.5 3.24e-01 1.67e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  80  1.5137658e+03 3.00e-01 1.13e+03  -1.0 3.24e+01  -2.0 1.50e-01 1.46e-01f  1
  81  1.4708367e+03 3.65e-01 2.03e+03  -1.0 1.16e+01  -1.5 3.56e-01 2.80e-01f  1
  82  1.4162975e+03 3.55e-01 6.58e+02  -1.0 2.49e+01  -2.0 1.20e-01 1.67e-01f  1
  83  1.3637059e+03 2.02e-01 5.05e+03  -1.0 9.27e+00  -1.6 2.17e-01 4.23e-01f  1
  84  1.3565899e+03 1.97e-01 2.16e+03  -1.0 3.81e+01  -2.1 8.50e-02 2.42e-02f  1
  85  1.2722675e+03 1.17e-01 3.90e+03  -1.0 1.05e+01  -1.6 6.37e-01 7.07e-01f  1
  86  1.2131402e+03 1.20e-01 1.66e+04  -1.0 2.27e+01  -2.1 1.61e-01 2.48e-01f  1
  87  1.1666310e+03 9.17e-02

 169  1.0917905e+02 6.79e-02 9.43e+01  -2.5 1.01e+01  -2.9 5.79e-01 4.33e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 170  1.0574799e+02 9.30e-02 6.53e+01  -2.5 2.87e+01  -3.4 1.56e-01 2.83e-01f  1
 171  1.0390491e+02 7.20e-02 8.45e+01  -2.5 1.12e+01  -2.9 2.25e-01 3.95e-01f  1
 172  1.0235718e+02 2.00e-02 5.24e+01  -2.5 4.14e+00  -2.5 4.97e-01 8.70e-01f  1
 173  1.0137788e+02 2.12e-02 7.98e+01  -2.5 1.29e+01  -3.0 5.71e-01 1.90e-01f  1
 174  9.8372774e+01 1.40e-01 4.92e+01  -2.5 3.88e+01  -3.5 1.34e-01 1.95e-01f  1
 175  9.5541990e+01 7.12e-02 5.43e+01  -2.5 1.34e+01  -3.0 2.34e-01 5.01e-01f  1
 176  9.3423831e+01 2.28e-02 2.94e+01  -2.5 5.13e+00  -2.6 8.23e-01 1.00e+00f  1
 177  9.0107597e+01 1.19e-01 2.73e+01  -2.5 1.55e+01  -3.1 4.40e-01 5.45e-01f  1
 178  8.8016497e+01 2.58e-02 1.29e+02  -2.5 5.55e+00  -2.7 2.58e-01 9.42e-01f  1
 179  8.4969194e+01 6.24e-02 5.76e+01  -2.5 1.74e+01  -3.1 4.97e-01 4.51e-01f  1
iter    objective    inf_pr 

 261  1.4273510e+01 7.48e-03 3.24e+01  -2.5 2.69e+00  -3.4 7.24e-01 3.88e-01h  1
 262  1.4228272e+01 1.97e-02 3.94e-02  -2.5 7.80e+00  -3.9 1.00e+00 1.00e+00f  1
 263  1.4141858e+01 1.08e-01 1.35e-01  -2.5 2.00e+01  -4.4 1.00e+00 1.00e+00h  1
 264  1.4086657e+01 2.21e-01 8.40e+00  -2.5 4.97e+01  -4.9 3.16e-01 4.56e-01h  1
 265  1.4050405e+01 8.80e-02 1.06e+01  -2.5 1.21e+01  -4.4 5.88e-01 7.21e-01h  1
 266  1.4010614e+01 1.52e-01 3.24e+00  -2.5 2.78e+01  -4.9 6.77e-01 6.74e-01h  1
 267  1.3985921e+01 5.11e-01 1.63e+00  -2.5 8.78e+01  -5.4 3.07e-01 2.89e-01h  1
 268  1.3950329e+01 3.54e-01 1.80e-01  -2.5 2.56e+01  -5.0 1.00e+00 1.00e+00h  1
 269  1.3943713e+01 4.57e-02 5.27e-02  -2.5 9.35e+00  -4.5 1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 270  1.3937128e+01 2.05e-01 5.74e-01  -2.5 5.77e+01  -5.0 3.13e-01 2.88e-01h  1
 271  1.3943596e+01 1.14e-01 3.18e-01  -2.5 7.42e+00  -3.7 4.48e-01 4.42e-01h  1
 272  1.3947416e+01 1.01e-01

 354  1.6420396e-02 1.01e-07 2.23e-03  -5.7 1.91e-03   0.1 1.00e+00 1.00e+00h  1
 355  1.6416749e-02 1.13e-06 2.05e+00  -5.7 3.37e-03  -0.4 1.00e+00 4.25e-01h  2
 356  1.6416389e-02 1.73e-06 5.26e+00  -5.7 8.71e-03  -0.9 7.70e-01 1.34e-01h  3
 357  1.6416223e-02 5.38e-07 5.29e-04  -5.7 1.57e-03  -0.5 1.00e+00 1.00e+00h  1
 358  1.6412929e-02 5.45e-07 1.32e-03  -5.7 1.32e-03  -0.9 1.00e+00 1.00e+00h  1
 359  1.6410632e-02 1.15e-06 1.27e-03  -5.7 3.08e-03  -1.4 1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 360  1.6403879e-02 9.94e-07 1.27e-03  -5.7 4.76e-03  -1.9 1.00e+00 1.00e+00h  1
 361  1.6384550e-02 1.20e-06 1.38e-03  -5.7 1.41e-02  -2.4 1.00e+00 1.00e+00h  1
 362  1.6332329e-02 3.54e-06 7.61e-04  -5.7 4.05e-02  -2.8 1.00e+00 1.00e+00h  1
 363  1.6194086e-02 2.96e-05 2.89e-03  -5.7 1.03e-01  -3.3 1.00e+00 1.00e+00h  1
 364  1.5862410e-02 1.94e-04 6.87e-01  -5.7 2.81e-01  -3.8 1.00e+00 9.20e-01h  1
 365  1.5292742e-02 7.67e-04

 447  7.4271179e-03 1.41e-02 6.21e-02  -5.7 4.05e+00  -7.3 5.90e-01 4.63e-01h  2
 448  7.4277566e-03 5.83e-06 6.71e-04  -5.7 2.38e-01  -5.0 1.00e+00 1.00e+00h  1
 449  7.4247908e-03 1.53e-05 2.89e-03  -5.7 4.20e-01  -5.5 1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 450  7.4234671e-03 8.30e-06 1.17e-03  -5.7 2.99e-01  -5.1 1.00e+00 1.00e+00h  1
 451  7.4214947e-03 2.42e-05 6.62e-01  -5.7 1.36e+00  -5.6 8.42e-01 3.30e-01h  2
 452  7.4206790e-03 8.96e-06 9.22e-04  -5.7 5.79e-01  -5.1 1.00e+00 1.00e+00H  1
 453  7.4163451e-03 7.57e-05 5.26e-03  -5.7 5.41e-01  -5.6 1.00e+00 1.00e+00h  1
 454  7.4143069e-03 5.42e-05 4.72e-01  -5.7 1.54e+00  -6.1 1.00e+00 4.02e-01h  2
 455  7.4154697e-03 5.39e-05 4.59e-01  -5.7 3.41e+00  -5.7 6.02e-02 3.39e-02h  3
 456  7.4138211e-03 3.27e-05 1.48e-01  -5.7 1.15e-01  -4.3 1.00e+00 3.95e-01h  2
 457  7.4139328e-03 2.35e-06 2.04e-04  -5.7 7.02e-02  -4.8 1.00e+00 1.00e+00h  1
 458  7.4134041e-03 1.77e-06

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 540  2.1337522e-04 1.98e-05 7.97e-02  -7.0 1.57e+00  -7.4 3.63e-01 2.30e-01h  2
 541  2.1333348e-04 1.19e-06 7.22e-04  -7.0 8.00e-02  -6.1 1.00e+00 1.00e+00h  1
 542  2.1318084e-04 3.11e-06 1.35e-03  -7.0 2.34e-01  -6.5 1.00e+00 1.00e+00h  1
 543  2.1303565e-04 1.06e-05 1.16e-01  -7.0 1.19e+00  -7.0 7.36e-01 2.81e-01h  2
 544  2.1297270e-04 1.89e-05 3.56e-03  -7.0 7.12e-01  -6.6 1.00e+00 1.00e+00h  1
 545  2.1269547e-04 1.00e-06 7.00e-03  -7.0 1.18e-01  -6.2 1.00e+00 1.00e+00h  1
 546  2.1257391e-04 3.78e-06 4.18e-03  -7.0 2.87e-01  -6.7 1.00e+00 1.00e+00h  1
 547  2.1223099e-04 2.93e-05 2.22e-03  -7.0 8.36e-01  -7.1 1.00e+00 1.00e+00h  1
 548  2.1131868e-04 2.06e-04 1.72e-02  -7.0 2.38e+00  -7.6 1.00e+00 1.00e+00h  1
 549  2.0900702e-04 1.23e-03 1.05e-01  -7.0 6.84e+00  -8.1 1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 550  2.0562352e-04 3.07e-03

ApplicationError: Solver (ipopt) did not exit normally

In [ ]:
#animate it
import matplotlib.pyplot as plt
import matplotlib.animation as ani
from IPython.display import HTML
%matplotlib inline

fig1, ax1 = plt.subplots(1,1,dpi=100) #create axes
ax1.set_aspect('equal')

xmax = np.max([m.q[n,P,'x'].value for n in range(1,N+1)])

def plot_robot(i,P,m,ax): #update function for animation
    ax.clear()
    ax.set_xlim([-1,xmax+1])
    ax.set_ylim([0,1.5])
    
    #plot body
    body_xb = m.q[i,P,'x'].value - 0.5*m.len[('body',1)]*cos(m.q[i,P,'thb'].value)
    body_yb = m.q[i,P,'y'].value - 0.5*m.len[('body',1)]*sin(m.q[i,P,'thb'].value)
    body_xf = m.q[i,P,'x'].value + 0.5*m.len[('body',1)]*cos(m.q[i,P,'thb'].value)
    body_yf = m.q[i,P,'y'].value + 0.5*m.len[('body',1)]*sin(m.q[i,P,'thb'].value)  
    ax.plot([body_xb,body_xf],[body_yb,body_yf],color='xkcd:black')
      
    #plot leg 1
    thA = m.q[i,P,'thl'].value
    leg1_xt = m.q[i,P,'x'].value
    leg1_yt = m.q[i,P,'y'].value
    leg1_xb = m.q[i,P,'x'].value + m.len[('leg',1)]*sin(thA)
    leg1_yb = m.q[i,P,'y'].value - m.len[('leg',1)]*cos(thA)
    ax.plot([leg1_xt,leg1_xb],[leg1_yt,leg1_yb],color='xkcd:red')
    
    #plot leg 2
    Lt = 0.5*m.len[('leg',1)] + m.q[i,P,'r'].value - 0.5*m.len[('leg',2)]
    Lb = 0.5*m.len[('leg',1)] + m.q[i,P,'r'].value + 0.5*m.len[('leg',2)]
    leg2_xt = m.q[i,P,'x'].value + Lt*sin(thA)
    leg2_yt = m.q[i,P,'y'].value - Lt*cos(thA)
    leg2_xb = m.q[i,P,'x'].value + Lb*sin(thA)
    leg2_yb = m.q[i,P,'y'].value - Lb*cos(thA)
    ax.plot([leg2_xt,leg2_xb],[leg2_yt,leg2_yb],color='xkcd:orange')
    
update = lambda i: plot_robot(i,P,m,ax1) #lambdify update function

animate = ani.FuncAnimation(fig1,update,range(1,N+1,3),interval = 50,repeat=True)

HTML(animate.to_html5_video()) #you need to convert the animation to HTML5 to embed it in the notebook
